In [3]:
import pandas as pd
from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from yahoofinancials import YahooFinancials
import wikipedia as wp
from concurrent.futures import ThreadPoolExecutor



In [28]:
token = '39L3MfwUDvhFO5we7eODz_Z-uXdrlSmV48djjnfwbzYL0Wz9ZpHcej7MzgsSZCvh-qsqop2OpZ4SdVhidJj80g=='
org = "my-org"
url = "http://3.109.41.82:8086"
bucket='raf'

In [5]:
df= pd.read_csv('/home/mehedi/Documents/docker_influxdb/DTCC_data/extracted_files/Commodities/CFTC_CUMULATIVE_COMMODITIES_2023_07_25.csv',index_col=0)
df

,Original Dissemination Identifier,Action type,Event type,Event timestamp,Amendment indicator,Asset Class,Product name,Cleared,Mandatory clearing indicator,Execution Timestamp,...,Option Style,Package indicator,Package transaction price,Package transaction price currency,Package transaction price notation,Package transaction spread,Package transaction spread currency,Package transaction spread notation,Physical delivery location-Leg 1,Delivery Type
Dissemination Identifier,,,,,,,,,,,,,,,,,,,,,
697754337,696534327.0,MODI,TRAD,2023-03-23T18:31:09Z,True,CO,Commodity:Agricultural:GrainsOilSeeds:Exotic,N,False,2023-03-23T13:07:27Z,...,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash
697754338,696533897.0,MODI,TRAD,2023-04-04T18:32:09Z,True,CO,Commodity:Agricultural:Softs:Exotic,N,False,2023-04-04T17:59:35Z,...,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash
697754339,696533431.0,MODI,TRAD,2023-03-24T20:40:31Z,True,CO,Commodity:Agricultural:Softs:Exotic,N,False,2023-03-24T17:52:44Z,...,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash
697754340,696534323.0,MODI,TRAD,2023-04-04T18:32:32Z,True,CO,Commodity:Agricultural:Softs:Exotic,N,False,2023-04-04T18:01:37Z,...,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash
697754341,696534325.0,MODI,TRAD,2023-04-13T16:55:39Z,True,CO,Commodity:Agricultural:Softs:Exotic,N,False,2023-04-13T16:48:33Z,...,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
697721209,NaN,NEWT,TRAD,2023-07-25T00:28:03Z,False,CO,Commodity:Agricultural:GrainsOilSeeds:Swap:Cash,N,False,2023-07-25T00:28:03Z,...,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
697721212,NaN,NEWT,TRAD,2023-07-25T00:30:41Z,NaN,CO,Commodity:Metals:NonPrecious:Swap:Cash,N,False,2023-07-25T00:30:41Z,...,NaN,True,9.9999999999,NaN,3.0,NaN,NaN,NaN,NaN,Cash
697721227,NaN,NEWT,TRAD,2023-07-25T00:28:23Z,False,CO,Commodity:Agricultural:GrainsOilSeeds:Swap:Cash,N,False,2023-07-25T00:28:23Z,...,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df.columns

Index(['Original Dissemination Identifier', 'Action type', 'Event type',
       'Event timestamp', 'Amendment indicator', 'Asset Class', 'Product name',
       'Cleared', 'Mandatory clearing indicator', 'Execution Timestamp',
       ...
       'Option Style', 'Package indicator', 'Package transaction price',
       'Package transaction price currency',
       'Package transaction price notation', 'Package transaction spread',
       'Package transaction spread currency',
       'Package transaction spread notation',
       'Physical delivery location-Leg 1', 'Delivery Type'],
      dtype='object', length=111)

In [25]:
data_type ='Commodities'
ticker = 'CFTC_CUMULATIVE_COMMODITIES_2023_07_25'


In [24]:
def point_crt(data_type, ticker, df, timestamp):
    field_list = df.columns.tolist()
    points = []
    
    for index, row in df.iterrows():
        point_builder = Point(data_type).tag("ticker", ticker).time(timestamp)
        
        for field_name in field_list:
            point_builder = point_builder.field(field_name, row[field_name])
        
        points.append(point_builder)
    
    return points

In [26]:
def write_to_influxdb(df, ticker, bucket, org):
    # Initialize the InfluxDB client
    client = InfluxDBClient(url=url, token=token)

    # Create a WriteAPI instance
    write_api = client.write_api(write_options=SYNCHRONOUS)

    # Iterate over each row in the DataFrame
    for index, row in df.iterrows():
        timestamp = pd.Timestamp(row['date']).to_pydatetime()
        point = point_crt(data_type, ticker, df, timestamp)
        write_api.write(bucket=bucket, org=org, record=point)
        # time.sleep(0.1)  # Optional: Sleep between each point to avoid rate-limiting
        print('done', ticker, 'for', timestamp)
    # Close the InfluxDB client
    client.close()

In [29]:
write_to_influxdb(df, ticker, bucket, org)

done CFTC_CUMULATIVE_COMMODITIES_2023_07_25 for 2023-07-25 00:00:00
done CFTC_CUMULATIVE_COMMODITIES_2023_07_25 for 2023-07-25 00:00:00
done CFTC_CUMULATIVE_COMMODITIES_2023_07_25 for 2023-07-25 00:00:00
done CFTC_CUMULATIVE_COMMODITIES_2023_07_25 for 2023-07-25 00:00:00
done CFTC_CUMULATIVE_COMMODITIES_2023_07_25 for 2023-07-25 00:00:00
done CFTC_CUMULATIVE_COMMODITIES_2023_07_25 for 2023-07-25 00:00:00
done CFTC_CUMULATIVE_COMMODITIES_2023_07_25 for 2023-07-25 00:00:00
done CFTC_CUMULATIVE_COMMODITIES_2023_07_25 for 2023-07-25 00:00:00
done CFTC_CUMULATIVE_COMMODITIES_2023_07_25 for 2023-07-25 00:00:00
done CFTC_CUMULATIVE_COMMODITIES_2023_07_25 for 2023-07-25 00:00:00
done CFTC_CUMULATIVE_COMMODITIES_2023_07_25 for 2023-07-25 00:00:00
done CFTC_CUMULATIVE_COMMODITIES_2023_07_25 for 2023-07-25 00:00:00
done CFTC_CUMULATIVE_COMMODITIES_2023_07_25 for 2023-07-25 00:00:00
done CFTC_CUMULATIVE_COMMODITIES_2023_07_25 for 2023-07-25 00:00:00
done CFTC_CUMULATIVE_COMMODITIES_2023_07_25 for 

KeyboardInterrupt: 